In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import pandas as pd
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
# from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle

/home/sebastian/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    """
    This function returns HOG-Featurs and a HOG-Img-Visualization. Input image needs to bee in grayscale.
    - < from skimage.feature import hog > needs to be imported!
    
    Input: gray_img, orien, pix_per_cell, cell_per_block, vis (default = False)
    
    Output: (if vis = True)-> features, hof_image | if vis = False -> features
    """
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                                  visualise=True, feature_vector=False)
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                       visualise=False, feature_vector=feature_vec)
        return features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, params):
    """
    Takes in a list of images, reads each image and converts it if needed.
    Calls get_hog_features() with vis=False, feature_vec=True
    
    Input:
    
    Output:
    
    """
    cspace = params['color_space']
    orient = params['orient']
    pix_per_cell = params['pix_per_cell']
    cell_per_block = params['cell_per_block']
    hog_channel = params['hog_channel']
    
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features

In [8]:
# Reading the Data 
car_images = glob.glob("vehicles/*/*.png")
non_car_images = glob.glob("non-vehicles/*/*.png")
cars = []
notcars = []
for image in car_images:
    #image = mpimg.imread(image)
    cars.append(image)
for image in non_car_images:
    #image = mpimg.imread(image)
    notcars.append(image)
#shuffle datta
cars = shuffle(cars)
notcars = shuffle(notcars)
# Reduce the sample size because HOG features are slow to compute
# The quiz evaluator times out after 13s of CPU time
#sample_size = 500
#cars = cars[0:sample_size]
#notcars = notcars[0:sample_size]

# Generator -- ? 
###############################################################################################
### TODO optimizate --- Logfile Report                                                 ########
params = {}
params['color_space'] = "YCrCb"   # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
params['orient'] = 11              # HOG orientations
params['pix_per_cell'] = 16        # HOG pixels per cell
params['cell_per_block'] = 2      # HOG cells per block
params['hog_channel'] = "ALL"    # Can be 0, 1, 2, or "ALL"
params['spatial_size'] = (16, 16) # Spatial binning dimensions
params['hist_bins'] = 16          # Number of histogram bins
params['spatial_feat'] = True     # Spatial features on or off
params['hist_feat'] = True        # Histogram features on or off
params['hog_feat'] = True # HOG features on

###############################################################################################
t=time.time()
car_features = extract_features(cars, params)
notcar_features = extract_features(notcars, params)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',params['orient'],'orientations',params['pix_per_cell'],
    'pixels per cell and', params['cell_per_block'],'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
training_time = round(t2-t, 2)
print(training_time, 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
pred_time = round(t2-t, 5)
print(pred_time, 'Seconds to predict', n_predict,'labels with SVC')

/home/sebastian/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


42.14 Seconds to extract HOG features...
Using: 11 orientations 16 pixels per cell and 2 cells per block
Feature vector length: 1188
2.0 Seconds to train SVC...
Test Accuracy of SVC =  0.9721
My SVC predicts:  [ 0.  1.  0.  0.  0.  1.  0.  0.  0.  0.]
For these 10 labels:  [ 0.  1.  0.  0.  0.  1.  0.  0.  0.  0.]
0.00109 Seconds to predict 10 labels with SVC


In [13]:
colorspace = params['color_space']   # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = params['orient']              # HOG orientations
pix_per_cell = params['pix_per_cell']       # HOG pixels per cell
cell_per_block = params['cell_per_block']     # HOG cells per block
hog_channel = params['hog_channel']    # Can be 0, 1, 2, or "ALL"
spatial_size = params['spatial_size'] # Spatial binning dimensions
hist_bins = params['hist_bins']         # Number of histogram bins
spatial_feat = params['spatial_feat']      # Spatial features on or off
hist_feat = params['hist_feat']         # Histogram features on or off
hog_feat = params['hog_feat'] # HOG features on

data_old = pd.read_csv("Hog_classifier/Hog_log.csv")
data_new = pd.DataFrame([[colorspace,orient,pix_per_cell,cell_per_block,hog_channel, 
                          training_time,pred_time,round(svc.score(X_test, y_test), 4)]], 
                        columns = ["colorspace", "orient", "pix_per_cell",
                               "cell_per_block","hog_channel", "training_time",
                               "Predict_time(10)","ACC"])
combined = data_old.append(data_new, ignore_index=True)
combined = combined.drop(["Unnamed: 0"], axis = 1)
combined.to_csv("Hog_classifier/Hog_log.csv")

In [9]:
import matplotlib.pyplot as plt
data_old = pd.read_csv("Hog_classifier/Hog_log.csv")
data_old

,Unnamed: 0,ACC,Predict_time(10),cell_per_block,colorspace,hog_channel,orient,pix_per_cell,training_time
0,0,0.9683,0.00423,2,RGB,0,9,8,6.89
1,1,0.9715,0.00488,2,RGB,1,9,9,4.43
2,2,0.9733,0.00364,2,HSV,2,9,8,6.67
3,3,0.9379,0.00341,2,HSV,1,9,8,10.75
4,4,0.9724,0.00304,2,HSV,0,9,8,5.90
5,5,0.9751,0.00173,2,RGB,2,9,8,6.16
6,6,0.9918,0.00496,2,RGB,ALL,9,8,22.46
7,7,0.9737,0.00230,2,LUV,0,9,8,4.01
8,8,0.9701,0.00212,2,LUV,1,9,8,3.82
9,9,0.9619,0.00198,2,LUV,2,9,8,4.37


In [6]:
# Save HOG Classifier to pickle file.
def save_classifier(pickle_file, svc, X_scaler, params):
    print('Saving classifier to pickle file...')
    try:
        with open(pickle_file, 'wb') as pfile:
            pickle.dump(
                {   'svc':svc, 
                    'scaler': X_scaler,

                    'color_space': params['color_space'],
                    'orient': params['orient'],
                    'pix_per_cell': params['pix_per_cell'],
                    'cell_per_block': params['cell_per_block'],
                    'hog_channel': params['hog_channel'],
                    'spatial_size': params['spatial_size'],
                    'hist_bins': params['hist_bins'],
                    'spatial_feat': params['spatial_feat'],
                    'hist_feat': params['hist_feat'],
                    'hog_feat': params['hog_feat']
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save classifier to', pickle_file, ':', e)


In [9]:
import pickle

save_classifier("Hog_classifier/hog-classifier_YCrCb116_shuffle_fulldata.pkl",svc,X_scaler,params)

Saving classifier to pickle file...


In [11]:
hog_c = pickle.load(open("Hog_classifier/hog-classifier.pkl","rb"))